In [1]:

# !pip install -q -U transformers accelerate bitsandbytes peft datasets safetensors sentencepiece evaluate huggingface_hub
!pip install -q -U "transformers>=4.38.0" "datasets>=2.18.0" "peft>=0.9.0" "bitsandbytes>=0.41.0" "accelerate>=0.28.0" "safetensors" "sentencepiece" "evaluate" "huggingface_hub"



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 89.6 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 29.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.8/485.8 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 87.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━

In [2]:
# imports + device
import os
import math
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer, BitsAndBytesConfig
)
from peft import (
    LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
)
from huggingface_hub import notebook_login
from tqdm.auto import tqdm
from huggingface_hub import login, HfFolder

print("Torch:", torch.__version__, " CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

login(token=HF_TOKEN) 
HfFolder.save_token(HF_TOKEN) 

2025-09-04 09:08:16.950980: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756976897.184497      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756976897.249148      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Torch: 2.6.0+cu124  CUDA available: True
GPU: Tesla T4


In [3]:
from huggingface_hub import HfApi
api = HfApi()

me = api.whoami(token=HF_TOKEN)
print("Token belongs to:", me.get("name") or me.get("user", {}).get("name") or me)

repo_id = "allout2726/type_summarized_git_diff_repo_node"
try:
    info = api.model_info(repo_id, token=HF_TOKEN)
    print("Repo found. Files:", info.siblings)     
except Exception as e:
    print("Could not access repo:", e)

Token belongs to: allout2726
Could not access repo: 404 Client Error. (Request ID: Root=1-68b9570f-42355394721054604eb6fd63;8f7fe134-449e-46d5-92da-89b169a94844)

Repository Not Found for url: https://huggingface.co/api/models/allout2726/type_summarized_git_diff_repo_node.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication


In [4]:
# Some globals
MODEL_NAME = "codellama/CodeLlama-7b-hf"
OUTPUT_DIR = "/kaggle/working/ccs_codellama7b_lora_summarised_node"
DATASET_ID = "rsh-raj/node-commits"
MAX_SEQ_LEN = 1024
PER_DEVICE_BATCH_SIZE = 1
NUM_EPOCHS =3
LEARNING_RATE = 3e-4
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
SEED = 42
torch.manual_seed(SEED)


In [5]:
#dataset
ds = load_dataset(DATASET_ID)

# ds["train"][0]
types_list_set = set()
for sample in ds["train"]:
    types_list_set.add(sample["type"])
types_list = list(types_list_set)
categories_str = ", ".join(sorted(types_list))

train.csv:   0%|          | 0.00/64.5M [00:00<?, ?B/s]

val.csv:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/39.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1400 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [6]:
# Prompt template used for finetuning purpose and aslo for inferencing
PROMPT_HEAD = (
    f"<s>[INST] <<SYS>>\n"
    f"You are a commit classifier based on commit message and summary of the code diff."
    f"Please classify the given commit into one of the these categories: {{categories}}\n"
    f"<</SYS>>\n\n"
)
PROMPT_HEAD = PROMPT_HEAD.format(categories=categories_str)
PROMPT_COMMIT_MESSAGE = f"- given commit message:\n{{message}}\n"
PROMPT_COMMIT_SUMMARISED_DIFF = f"- given summary of the code diff: \n{{diff}}\n"

def applyTemplate(sample):
    return {
        "prompt_commit_message": PROMPT_COMMIT_MESSAGE.format(message=sample["masked_commit_message"]),
        "prompt_commit_summarised_diff": PROMPT_COMMIT_SUMMARISED_DIFF.format(diff=sample["summarized_git_diff"]),
        "response": f"[/INST] {sample['type']} </s>",
    }
print(PROMPT_HEAD[:400])
# def build_prompt(message, diff):
#     return PROMPT_HEAD + PROMPT_COMMIT_MESSAGE.format(message=message) + PROMPT_COMMIT_DIFF.format(diff=diff)
# temp = applyTemplate(ds["train"][0])
# print(tmp["prompt_commit_message"][:200])
# print("response:", tmp["response"])


<s>[INST] <<SYS>>
You are a commit classifier based on commit message and summary of the code diff.Please classify the given commit into one of the these categories: assert, benchmark, bootstrap, buffer, build, cluster, console, crypto, debugger, deps, dgram, dns, doc, errors, esm, events, fs, http, inspector, lib, meta, module, net, path, process, quic, readline, repl, report, src, stream, test, 


In [7]:
# Map template i.e apply template to all the dataset
for i in ds.keys():
    ds[i] = ds[i].map(lambda x: applyTemplate(x))
print("Columns after template:", ds["train"].column_names)


Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Columns after template: ['hash', 'date', 'author', 'commit_message', 'is_merge', 'masked_commit_message', 'type', 'git_diff', 'summarized_git_diff', 'prompt_commit_message', 'prompt_commit_summarised_diff', 'response']


In [8]:
# Tokenizer load & adjustments
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=False, use_fast=False)
# ensure special tokens exist
if tokenizer.eos_token is None:
    tokenizer.add_special_tokens({"eos_token": "</s>"})
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token




tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [9]:
# Tokenization helper to add labels
def tokenize_add_label(sample, tokenizer=tokenizer, max_seq_len=MAX_SEQ_LEN):
    prompt_head_ids = tokenizer.encode(PROMPT_HEAD, add_special_tokens=False)
    message_ids = tokenizer.encode(sample["prompt_commit_message"], max_length=64, truncation=True, add_special_tokens=False)
    response_ids = tokenizer.encode(sample["response"], max_length=20, truncation=True, add_special_tokens=False)
    remaining = max_seq_len - (len(prompt_head_ids) + len(message_ids) + len(response_ids))
    if remaining < 0:
        message_ids = message_ids[:32]
        remaining = max_seq_len - (len(prompt_head_ids) + len(message_ids) + len(response_ids))
        if remaining < 0:
            response_ids = response_ids[:8]
            remaining = max_seq_len - (len(prompt_head_ids) + len(message_ids) + len(response_ids))
            remaining = max(0, remaining)
    diff_ids = tokenizer.encode(sample["prompt_commit_summarised_diff"], max_length=remaining, truncation=True, add_special_tokens=False)
    seq = prompt_head_ids + message_ids + diff_ids + response_ids
    pad_len = max_seq_len - len(seq)
    if pad_len > 0:
        seq += [tokenizer.eos_token_id] * pad_len
    attention_mask = [1]*max_seq_len
    labels = [-100] * (len(prompt_head_ids) + len(message_ids) + len(diff_ids)) + response_ids + [-100] * pad_len
    return {"input_ids": seq, "attention_mask": attention_mask, "labels": labels}

# Apply tokenization to all dataset
# for i in ds.keys():
for i in ds.keys():
    print("Tokenizing key:", i)
    ds[i] = ds[i].map(lambda x: tokenize_add_label(x), remove_columns=ds[i].column_names)
    ds[i].set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
print("Tokenization finished.")
ds["train"][0]


Tokenizing key: train


Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

Tokenizing key: validation


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Tokenizing key: test


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Tokenization finished.


{'input_ids': tensor([    1, 29961, 25580,  ...,     2,     2,     2]),
 'attention_mask': tensor([1, 1, 1,  ..., 1, 1, 1]),
 'labels': tensor([-100, -100, -100,  ..., -100, -100, -100])}

In [10]:
# BitsAndBytes / quantization config + load model
from transformers import BitsAndBytesConfig

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=torch.float16
# )
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=False,
    dtype=torch.float16
)

print("Model loaded.")


config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Model loaded.


In [11]:
# Prepare our model and apply LoRA (PEFT)
model = prepare_model_for_kbit_training(model)

target_modules = ["q_proj", "v_proj"]

peft_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=target_modules,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)


In [12]:
# Training arguments & trainer
from transformers import default_data_collator

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_BATCH_SIZE,
    save_strategy="epoch",
    num_train_epochs=3,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_strategy="steps",
    logging_steps=50,
    gradient_accumulation_steps=4,
    save_total_limit=2,
    remove_unused_columns=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    data_collator=default_data_collator
)
print("Trainer created.")


Trainer created.


In [13]:
# Finetune the model and save
trainer.train()
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
repo_id="allout2726/type_summarized_git_diff_repo_node"
model.push_to_hub(repo_id, use_auth_token=HF_TOKEN)
tokenizer.push_to_hub(repo_id, use_auth_token=HF_TOKEN)
print("Training finished and LoRA adapter saved to", OUTPUT_DIR)


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,0.975900
100,0.147500
150,0.108400
200,0.089900
250,0.073400
300,0.091000
350,0.077600
400,0.061800
450,0.046000
500,0.056400


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/transformers/u

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

...

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpnz_ni0kf/tokenizer.model      :  94%|#########3|  469kB /  500kB            

Training finished and LoRA adapter saved to /kaggle/working/ccs_codellama7b_lora_summarised_node


In [14]:
# !pip install -q -U "transformers>=4.38.0" "datasets>=2.18.0" "peft>=0.9.0" "bitsandbytes>=0.41.0" "accelerate>=0.28.0" "safetensors" "sentencepiece" "evaluate" "huggingface_hub"
# import os
# import math
# import torch
# from datasets import load_dataset
# from transformers import (
#     AutoTokenizer, AutoModelForCausalLM,
#     TrainingArguments, Trainer, BitsAndBytesConfig
# )
# from peft import (
#     LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
# )
# from huggingface_hub import notebook_login
# from tqdm.auto import tqdm
#inferencing the model on validation dataset
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Load base again
base = AutoModelForCausalLM.from_pretrained(MODEL_NAME, quantization_config=bnb_config, device_map="auto", torch_dtype=torch.float16)
base = PeftModel.from_pretrained(base, OUTPUT_DIR, device_map="auto")  # load LoRA adapter
base.eval()
import re
# List of all possible types for classification
# TYPES_LIST = ["docs","perf","style","refactor","feat","fix","test","ci","build","chore"]
def build_prompt(message, diff):
    return PROMPT_HEAD + PROMPT_COMMIT_MESSAGE.format(message=message) + PROMPT_COMMIT_SUMMARISED_DIFF.format(diff=diff)

# def extractType(text):
#     lowered = text.lower()
#     for t in TYPES_LIST:
#         if (" " + t + " ") in (" " + lowered + " "):
#             return t
#     toks = lowered.replace("[/inst]","").replace("</s>","").strip().split()
#     for w in toks[:10]:
#         if w in TYPES_LIST:
#             return w
#     return None

def extractType(text):
    lowered = text.lower()

    matches = list(re.finditer(r'\[/inst\]', lowered, flags=re.IGNORECASE))
    if matches:
        tail = lowered[matches[-1].end():]  # substring after last [/INST]
    else:
        mid = max(0, len(lowered) // 2)
        tail = lowered[mid:]

    tail = re.split(r'<\/?s>', tail)[0]

    # Trim whitespace at edges
    tail = tail.strip()

    for t in types_list:
        if re.search(r'\b' + re.escape(t) + r'\b', tail):
            return t

    tokens = re.split(r'\s+', tail)[:8]
    for tok in tokens:
        cand = re.sub(r'[^a-zA-Z]', '', tok).lower()
        if cand in types_list:
            return cand

    for t in types_list:
        if re.search(r'\b' + re.escape(t) + r'\b', lowered):
            return t

    return None

# evaluate on validation
test_ds = load_dataset(DATASET_ID)["validation"]# right now inferencing on just 100 samples
correct = 0
total = 0
for sample in tqdm(test_ds):
    prompt = build_prompt(sample["masked_commit_message"], sample["summarized_git_diff"])
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LEN).to(base.device)
    with torch.no_grad():
        out = base.generate(**inputs, max_new_tokens=16, do_sample=False, temperature=0.0, eos_token_id=tokenizer.eos_token_id,pad_token_id=tokenizer.eos_token_id,)
    text_output=tokenizer.decode(out[0], skip_special_tokens=False)
    pred = extractType(text_output)
    actual_type = sample["type"].lower()
    # print(f"output :",text_output)
    print(f"pred: ",pred)
    print(f"actual: ",actual_type)

    if pred == actual_type:
        correct += 1
    total += 1

print(f"Test accuracy: {100*correct/total:.2f}% ({correct}/{total})")


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


pred:  lib
actual:  crypto
pred:  doc
actual:  doc
pred:  lib
actual:  src
pred:  doc
actual:  doc
pred:  deps
actual:  deps
pred:  test
actual:  test
pred:  errors
actual:  src
pred:  dns
actual:  dns
pred:  build
actual:  doc
pred:  path
actual:  doc
pred:  lib
actual:  lib
pred:  timers
actual:  quic
pred:  meta
actual:  meta
pred:  doc
actual:  doc
pred:  doc
actual:  doc
pred:  doc
actual:  doc
pred:  doc
actual:  doc
pred:  tools
actual:  tools
pred:  errors
actual:  test
pred:  deps
actual:  test
pred:  worker
actual:  src
pred:  lib
actual:  lib
pred:  deps
actual:  assert
pred:  timers
actual:  lib
pred:  url
actual:  src
pred:  buffer
actual:  src
pred:  lib
actual:  src
pred:  doc
actual:  repl
pred:  process
actual:  process
pred:  crypto
actual:  tls
pred:  deps
actual:  deps
pred:  doc
actual:  doc
pred:  test
actual:  test
pred:  deps
actual:  src
pred:  build
actual:  build
pred:  tls
actual:  tls
pred:  deps
actual:  test
pred:  benchmark
actual:  buffer
pred:  lib
act

In [15]:
# import os
import re
import torch
from tqdm.auto import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

repo_id = "allout2726/type_summarized_git_diff_repo_node"  



login(HF_TOKEN)
print("Authenticated to Hugging Face via Kaggle Secrets.")


# 1) Load tokenizer from the Hub (this picks tokenizer that you pushed)
print("Loading tokenizer from:", repo_id)
tokenizer = AutoTokenizer.from_pretrained(repo_id, use_auth_token=HF_TOKEN)

# 2) Load base model (quantized) and then load the LoRA adapter from the Hub repo
print("Loading base model:", MODEL_NAME)
base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    dtype=torch.float16,
    use_auth_token=HF_TOKEN
)

print("Loading LoRA adapter from Hub repo:", repo_id)
base = PeftModel.from_pretrained(
    base,
    repo_id,
    device_map="auto",
    dtype=torch.float16,
    use_auth_token=HF_TOKEN
)

base.eval()
print("Model + adapter loaded. Device:", next(base.parameters()).device)

# helper to build prompt 
def build_prompt(message, diff):
    return PROMPT_HEAD + PROMPT_COMMIT_MESSAGE.format(message=message) + PROMPT_COMMIT_SUMMARISED_DIFF.format(diff=diff)

# TYPES_LIST = ["docs","perf","style","refactor","feat","fix","test","ci","build","chore"]

def extractType(text):
    lowered = text.lower()
    matches = list(re.finditer(r'\[/inst\]', lowered, flags=re.IGNORECASE))
    if matches:
        tail = lowered[matches[-1].end():]
    else:
        return None
        mid = max(0, len(lowered) // 2)
        tail = lowered[mid:]
    tail = re.split(r'<\/?s>', tail)[0].strip()
    for t in types_list:
        if re.search(r'\b' + re.escape(t) + r'\b', tail):
            return t
    tokens = re.split(r'\s+', tail)[:8]
    for tok in tokens:
        cand = re.sub(r'[^a-zA-Z]', '', tok).lower()
        if cand in types_list:
            return cand
    for t in types_list:
        if re.search(r'\b' + re.escape(t) + r'\b', lowered):
            return t
    return None

test_ds = load_dataset(DATASET_ID)["test"]
correct = 0
total = 0

for sample in tqdm(test_ds):
    prompt = build_prompt(sample["masked_commit_message"], sample["summarized_git_diff"])
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LEN).to(next(base.parameters()).device)
    with torch.no_grad():
        out = base.generate(
            **inputs,
            max_new_tokens=16,
            do_sample=False,
            temperature=0.0,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )
    text_output = tokenizer.decode(out[0], skip_special_tokens=False)
    # print(text_output)
    # print()
    pred = extractType(text_output)
    actual_type = sample["type"].lower()

    print("pred: ", pred)
    print("actual:", actual_type)

    if pred == actual_type:
        correct += 1
    total += 1

print(f"Test accuracy: {100*correct/total:.2f}% ({correct}/{total})")


Authenticated to Hugging Face via Kaggle Secrets.
Loading tokenizer from: allout2726/type_summarized_git_diff_repo_node


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:1010: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

...

adapter_config.json:   0%|          | 0.00/859 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

Model + adapter loaded. Device: cuda:0


  0%|          | 0/400 [00:00<?, ?it/s]

pred:  repl
actual: repl
pred:  tools
actual: tools
pred:  deps
actual: deps
pred:  doc
actual: meta
pred:  doc
actual: doc
pred:  lib
actual: lib
pred:  doc
actual: doc
pred:  test
actual: test
pred:  doc
actual: doc
pred:  build
actual: build
pred:  tools
actual: doc
pred:  test
actual: meta
pred:  test
actual: test
pred:  test
actual: test
pred:  doc
actual: doc
pred:  test
actual: test
pred:  deps
actual: deps
pred:  test
actual: test
pred:  test
actual: test
pred:  doc
actual: meta
pred:  src
actual: crypto
pred:  util
actual: util
pred:  test
actual: test
pred:  benchmark
actual: benchmark
pred:  doc
actual: doc
pred:  test
actual: test
pred:  deps
actual: deps
pred:  test
actual: test
pred:  doc
actual: doc
pred:  quic
actual: quic
pred:  build
actual: build
pred:  test
actual: test
pred:  test
actual: test
pred:  net
actual: net
pred:  doc
actual: doc
pred:  src
actual: src
pred:  test
actual: test
pred:  util
actual: util
pred:  doc
actual: doc
pred:  tools
actual: tools
pred:

In [16]:
!ls -la "/kaggle/working/ccs_codellama7b_lora"

ls: cannot access '/kaggle/working/ccs_codellama7b_lora': No such file or directory


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
